In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, roc_curve, precision_recall_curve, auc

Graph Split Function

In [ ]:
def train_test_split_graph(G, train_frac=0.8, test_frac=0.1, seed=42):
    np.random.seed(seed)
    edges = list(G.edges())
    np.random.shuffle(edges)
    m = len(edges)
    n_train = int(train_frac * m)
    n_test = int(test_frac * m)

    train_edges = edges[:n_train]
    test_pos = edges[n_train:n_train + n_test]

    G_train = nx.Graph()
    G_train.add_nodes_from(G.nodes())
    G_train.add_edges_from(train_edges)

    non_edges = list(nx.non_edges(G_train))
    np.random.shuffle(non_edges)
    test_neg = non_edges[:n_test]

    return G_train, test_pos, test_neg